In [13]:
import tensorflow as tf

from utils import HelperUtils, Configuration

In [15]:
data_dir = HelperUtils.load_data('./data/big.zip', 'update_only_new_data')
HelperUtils.convert_dataset(data_dir)

1748336640/1748324698 [==============================] - 2s 0us/step


In [16]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=Configuration.validation_split,
    subset="training",
    seed=123,
    image_size=(Configuration.img_height, Configuration.img_width),
    batch_size=Configuration.batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=Configuration.validation_split,
    subset="validation",
    seed=123,
    image_size=(Configuration.img_height, Configuration.img_width),
    batch_size=Configuration.batch_size)

class_names = train_ds.class_names

num_classes = len(class_names)

# train_ds = train_ds.shuffle(500).batch(batch_size)
# val_ds = val_ds.shuffle(500).batch(batch_size)

Found 77638 files belonging to 468 classes.
Using 62111 files for training.
Found 77638 files belonging to 468 classes.
Using 15527 files for validation.


In [21]:
model = tf.keras.models.load_model(f'{Configuration.models_location}/initial_model.h5')

opt = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 180, 180, 3)       0         
                                                                 
 conv2d_3 (Conv2D)           (None, 180, 180, 16)      448       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 90, 90, 16)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 90, 90, 32)        4640      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 45, 45, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 45, 45, 64)       

In [22]:
model.fit(train_ds, validation_data=val_ds, epochs=Configuration.epochs)
model.save(f'{Configuration.models_location}/update_only_new_data_model.h5')

Epoch 1/10
1941/1941 [==============================] - 44s 23ms/step - loss: 5.4631 - accuracy: 0.0365 - val_loss: 4.9146 - val_accuracy: 0.0725
Epoch 2/10
1941/1941 [==============================] - 35s 18ms/step - loss: 4.3038 - accuracy: 0.1307 - val_loss: 3.8235 - val_accuracy: 0.1893
Epoch 3/10
1941/1941 [==============================] - 35s 18ms/step - loss: 3.3417 - accuracy: 0.2567 - val_loss: 3.1812 - val_accuracy: 0.2830
Epoch 4/10
1941/1941 [==============================] - 35s 18ms/step - loss: 2.6132 - accuracy: 0.3780 - val_loss: 2.7344 - val_accuracy: 0.3680
Epoch 5/10
1941/1941 [==============================] - 35s 18ms/step - loss: 2.0544 - accuracy: 0.4841 - val_loss: 2.3281 - val_accuracy: 0.4480
Epoch 6/10
1941/1941 [==============================] - 35s 18ms/step - loss: 1.6037 - accuracy: 0.5813 - val_loss: 2.1229 - val_accuracy: 0.4985
Epoch 7/10
1941/1941 [==============================] - 35s 18ms/step - loss: 1.2661 - accuracy: 0.6621 - val_loss: 1.9314 -

In [ ]:
HelperUtils.test(model, class_names)